In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
%cd gdrive/My\ Drive/Colab\ Notebooks/word_generator/

/content/gdrive/My Drive/Colab Notebooks/word_generator


In [1]:
!git add fables_aesop.txt kids_stories.txt word_gen_MLM.ipynb word_gen_emb_pretrained.ipynb

fatal: not a git repository (or any of the parent directories): .git


## Process text

In [0]:
import string

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# turn a doc into clean tokens
def clean_doc(doc):
	# replace '--' with a space ' '
	doc = doc.replace('--', ' ')
	# split into tokens by white space
	tokens = doc.split()
	# remove punctuation from each token
	table = str.maketrans('', '', string.punctuation)
	tokens = [w.translate(table) for w in tokens]
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# make lower case
	tokens = [word.lower() for word in tokens]
	return tokens

# save tokens to file, one dialog per line
def save_doc(lines, filename):
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()


In [4]:

# load document
in_filename = 'fables_aesop.txt'
doc = load_doc(in_filename)
print(doc[:200])

# clean document
tokens = clean_doc(doc)
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

# organize into sequences of tokens
length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
	# select sequence of tokens
	seq = tokens[i-length:i]
	# convert into a line
	line = ' '.join(seq)
	# store
	sequences.append(line)
print('Total Sequences: %d' % len(sequences))

# save sequences to file
out_filename = 'fables_aesop_sequences.txt'
save_doc(sequences, out_filename)

THE WOLF AND THE LAMB


ONE day a Wolf and a Lamb happened to come at the same time to drink
from a brook that ran down the side of the mountain.

The Wolf wished very much to eat the Lamb, but meetin
['the', 'wolf', 'and', 'the', 'lamb', 'one', 'day', 'a', 'wolf', 'and', 'a', 'lamb', 'happened', 'to', 'come', 'at', 'the', 'same', 'time', 'to', 'drink', 'from', 'a', 'brook', 'that', 'ran', 'down', 'the', 'side', 'of', 'the', 'mountain', 'the', 'wolf', 'wished', 'very', 'much', 'to', 'eat', 'the', 'lamb', 'but', 'meeting', 'her', 'as', 'he', 'did', 'face', 'to', 'face', 'he', 'thought', 'he', 'must', 'find', 'some', 'excuse', 'for', 'doing', 'so', 'so', 'he', 'began', 'by', 'trying', 'to', 'pick', 'a', 'quarrel', 'and', 'said', 'angrily', 'dare', 'you', 'come', 'to', 'my', 'brook', 'and', 'muddy', 'the', 'water', 'so', 'that', 'i', 'cannot', 'drink', 'it', 'what', 'do', 'you', 'the', 'lamb', 'very', 'much', 'alarmed', 'said', 'gently', 'do', 'not', 'see', 'how', 'it', 'can', 'be', 'that

## get training data

Read the vector from glove file

In [0]:
import numpy as np

def read_glove_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1

    return words_to_index, index_to_words, word_to_vec_map

In [0]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('glove.6B.50d.txt')

In [0]:
# GRADED FUNCTION: sentences_to_indices

def sentences_to_indices(X, word_to_index, index_to_word, word_to_vec_map, max_len):
    """
    Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences.
    The output shape should be such that it can be given to `Embedding()` (described in Figure 4). 
    
    Arguments:
    X -- array of sentences (strings), of shape (m, 1)
    word_to_index -- a dictionary containing the each word mapped to its index
    max_len -- maximum number of words in a sentence. You can assume every sentence in X is no longer than this. 
    
    Returns:
    X_indices -- array of indices corresponding to words in the sentences from X, of shape (m, max_len)
    """
    
    m = X.shape[0]                                   # number of training examples
    vec_len = word_to_vec_map['baseball'].shape[0]
    # import pdb; pdb.set_trace()
    ### START CODE HERE ###
    # Initialize X_indices as a numpy matrix of zeros and the correct shape (≈ 1 line)
    X_indices = np.zeros( shape = (m, max_len))
    
    for i in range(m):                               # loop over training examples
        
        # Convert the ith training sentence in lower case and split is into words. You should get a list of words.
        # sentence_words = list(map(lambda x : x.lower() , X[i].split(" ")))
        sentence_words = [i.lower() for i in X[i].split()]
        # Initialize j to 0
        j = 0
        
        # Loop over the words of sentence_words
        for w in sentence_words:
            # Set the (i,j)th entry of X_indices to the index of the correct word.
            try:
              X_indices[i, j] = word_to_index[w]
            except KeyError:
              print(w + " doesnt have index, new entry created")
              
              vocab_size = len(word_to_index) + 1
              word_to_index[w] = vocab_size
              X_indices[i, j] = vocab_size
              word_to_vec_map[w] = np.random.rand(vec_len)
              index_to_word[vocab_size] = w 
              continue
            # Increment j to j + 1
            j += 1
    ### END CODE HERE ###
    # pdb.set_trace()
    return X_indices

In [8]:
X1 = np.array(["funny lol", "lets play baseball", "food is ready for you"])
X1_indices = sentences_to_indices(X1,word_to_index, index_to_word, word_to_vec_map, max_len = 5)
print("X1 =", X1)
print("X1_indices =", X1_indices)

X1 = ['funny lol' 'lets play baseball' 'food is ready for you']
X1_indices = [[155345. 225122.      0.      0.      0.]
 [220930. 286375.  69714.      0.      0.]
 [151204. 192973. 302254. 151349. 394475.]]


In [9]:
from numpy import array
from pickle import dump
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Bidirectional 


In [0]:
# load
in_filename = 'fables_aesop_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
lines = np.array(lines)
length = 50 + 1


In [11]:
print(lines[:5])

['the wolf and the lamb one day a wolf and a lamb happened to come at the same time to drink from a brook that ran down the side of the mountain the wolf wished very much to eat the lamb but meeting her as he did face to face he'
 'wolf and the lamb one day a wolf and a lamb happened to come at the same time to drink from a brook that ran down the side of the mountain the wolf wished very much to eat the lamb but meeting her as he did face to face he thought'
 'and the lamb one day a wolf and a lamb happened to come at the same time to drink from a brook that ran down the side of the mountain the wolf wished very much to eat the lamb but meeting her as he did face to face he thought he'
 'the lamb one day a wolf and a lamb happened to come at the same time to drink from a brook that ran down the side of the mountain the wolf wished very much to eat the lamb but meeting her as he did face to face he thought he must'
 'lamb one day a wolf and a lamb happened to come at the same time to d

In [12]:
# integer encode sequences of words
#okenizer = Tokenizer()
#tokenizer.fit_on_texts(lines)

#array([   15,     1,   306, 10270,    28,     7,   363,   225,  4604,....])
sequences = sentences_to_indices(lines,word_to_index, index_to_word, word_to_vec_map, max_len = length)
# vocabulary size
vocab_size = len(word_to_index) + 1  
X, y = sequences[:,:-1], sequences[:,-1]
#y = to_categorical(y, num_classes=vocab_size) eats all memory use sparse_categorical_crossentropy
seq_length = X.shape[1]

famishing doesnt have index, new entry created
storkking doesnt have index, new entry created
grayear doesnt have index, new entry created
whitewhisker doesnt have index, new entry created
gayly doesnt have index, new entry created
unsuspicious doesnt have index, new entry created
capered doesnt have index, new entry created
selfimportance doesnt have index, new entry created
dainties doesnt have index, new entry created
manyfeathered doesnt have index, new entry created
oneeyed doesnt have index, new entry created
repine doesnt have index, new entry created
filberts doesnt have index, new entry created
illmannered doesnt have index, new entry created
cowhides doesnt have index, new entry created
fairtime doesnt have index, new entry created
wellproved doesnt have index, new entry created
dinnerless doesnt have index, new entry created
miry doesnt have index, new entry created
selfinterest doesnt have index, new entry created
evildisposed doesnt have index, new entry created
faultfindi

In [13]:
len(word_to_vec_map)

400303

In [14]:
word_to_index['mermaid']

241527

## Pretrain embedding layer

In [0]:

def pretrained_embedding_layer(word_to_vec_map, word_to_index, trainable = False):
    """
    Creates a Keras Embedding() layer and loads in pre-trained GloVe 50-dimensional vectors.
    
    Arguments:
    word_to_vec_map -- dictionary mapping words to their GloVe vector representation.
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    embedding_layer -- pretrained layer Keras instance
    """
    
    vocab_len = len(word_to_index) + 1                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map["cucumber"].shape[0]      # define dimensionality of your GloVe word vectors (= 50)
    
    ### START CODE HERE ###
    # Initialize the embedding matrix as a numpy array of zeros of shape (vocab_len, dimensions of word vectors = emb_dim)
    emb_matrix = np.zeros(shape=(vocab_len,emb_dim))
    
    # Set each row "index" of the embedding matrix to be the word vector representation of the "index"th word of the vocabulary
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]

    # Define Keras embedding layer with the correct output/input sizes, make it non-trainable. Use Embedding(...). Make sure to set trainable=False. 
    embedding_layer = Embedding(vocab_len,emb_dim,trainable=trainable)
    
    ### END CODE HERE ###

    # Build the embedding layer, it is required before setting the weights of the embedding layer. Do not modify the "None".
    embedding_layer.build((None,))
    
    # Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [0]:
embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
print("weights[0][1][3] =", embedding_layer.get_weights()[0][1][3])

NameError: ignored

In [0]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import Adagrad
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint

In [0]:

# define model
model = Sequential()
# in : (batch, input_lengh) , [12,3,2,3,4,5,12,..., input_lentgh]

model.add(pretrained_embedding_layer(word_to_vec_map,word_to_index,trainable = False))
model.add(Bidirectional(LSTM(100, return_sequences=True)) )
model.add(Bidirectional(LSTM(100)) )
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())
# compile model
opt = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
# opt = SGD(learning_rate=0.001, momentum=0.0)
# opt = Adagrad(learning_rate=0.001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

filepath="weights-improvement.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
# fit model
model.fit(X, y, batch_size=128, epochs=100, callbacks=callbacks_list)

# save the model to file
model.save('model_embpre.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))

Reload and continue training

In [0]:
from tensorflow.keras.models import load_model

filepath="weights-improvement.hdf5"
model = load_model(filepath)
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
# fit model
model.fit(X, y, batch_size=128, epochs=100, callbacks=callbacks_list)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 88749 samples
Epoch 1/100
54272/88749 [=================>............] - ETA: 1:32 - loss: 3.4743 - acc: 0.2600

fatal: not a git repository (or any of the parent directories): .git
